# Sessão de pré-processamento na base credit-data 

## Import e carregamento do dataset

In [54]:
# Import das libs a serem utilizadas
import pandas as pd
import numpy

In [62]:
data = pd.read_csv('credit-data.csv')

In [56]:
# Verificando as estatisticas no dataset
# Na verificação é constada as porcentagens de 25% / 50% / 75%
# Essas porcentagens indicam respectivamente: 1º Quartil / Mediana / 3º Quartil

data.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [57]:
# Ao rodar o comando describe
# Foi notado que a base apresenta inconsistência na coluna de idade por apresentar valores negativos
# Com isso poderemos ter alguma problema de associação ou padrão encontrado pelo algoritmo com essas pessoas

## Verificação de valores inconsistentes (Constatação dos clientes que estão com idades negativas)

In [63]:
# Verificação dos cliente que estão com idades negativas
data.loc[data['age'] < 0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


## Estratégias para tratamento de valores inconsistentes

### Apagar toda a coluna (Não é tão recomendado, exceto em alguns casos)

In [7]:
# Primeiro passa a coluna que deseja remover
# Segundo parâmetro com valor 1 indica que quer apagar a coluna inteira
# Terceiro parâmetro indica TRUE indica que não haverá retorno e será realiado a função no próprio dataset
data.drop('age', 1, inplace=True)

### Apagar somente os registros que estão inconsistentes (Não é tão recomendado, exceto em alguns casos)

#### Poderá haver registros importantes com outros atributos que influenciará no algoritmo

In [12]:
# Primeiro passa os indices dos registros que deseja remover
# Segunda parâmetro indica TRUE indica que não haverá retorno e será realiado a função no próprio dataset
data.drop(data[data.age < 0].index, inplace=True)

### Preencher os valores manualmente

#### Essa estratégia se torna complicada dependendo do número de registro

### Preencher os valores pela média (Estratégia mais interessante)

In [58]:
# Não pode retirar a média do dataset levando em consideração os registros inconsistentes
# Desta forma, abaixo estamos desconsiderando os registros sujos do dataset e trabalhando com a média correta.
data['age'][data.age > 0].mean()

40.92770044906149

## Aplicando a correção nos registros inconsistentes

### Iremos substituir pela média desconsiderando aqueles valores sujos

In [65]:
# Primeiro valor da matriz é a condição
# Segundo valor da matriz indica que iremos substituir naquela coluna

data.loc[data.age < 0, 'age'] = data['age'][data.age > 0].mean()

# Não possuímos mais os valores negativos
# As linhas que foram modificadas foram as:
# 15 | 21 | 26

data.loc[15:26]

,clientid,income,age,loan,default
15,16,50501.726689,40.927700,3977.287432,0
16,17,43548.654711,39.574530,3935.544453,0
17,18,43378.175194,60.848318,3277.737553,0
18,19,20542.365073,61.690571,3157.442290,0
19,20,58887.357549,26.076093,4965.516066,0
20,21,23000.784002,31.761354,1148.118057,0
21,22,32197.620701,40.927700,4244.057136,0
22,23,23329.319414,48.576975,222.622299,0
23,24,27845.800894,51.970624,4959.921226,0
24,25,65301.984029,48.840922,5465.267886,0


## Separação das colunas (previsores e classe)

In [66]:
# Obtendo a separação das colunas previsores e classe

# 1º obteremos as colunas previsores as quais são as caracteristicas dos registros
# 2º Obtemos todos os registros das colunas INCOME a LOAN
# OBS: Porque desconsideramos o id? Devido à ML obter o máximo de generalização
X = data.iloc[:, 1:4].values

# 3º Obter a ultima coluna do nosso dataset a qual possui o rótulo com todos os registros
Y = data.iloc[:, -1].values

## Verificação de valores faltantes

## Como verificar se é Null ou NaN ou outro?

In [67]:
# Neste exemplo, podemos verificar a quantidade de faltantes por atributo
data.isnull().sum()

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [68]:
# Neste exemplo, podemos verificar diretamente o atributo se há faltantes
# Porém, fica inviável caso você possua vários atributos

pd.isnull(data['age'])

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: age, Length: 2000, dtype: bool

In [69]:
# Neste exemplo, é uma maneira de localizar melhor que a anterior
# Porém, fica também inviável com múltiplas colunas

data.loc[pd.isnull(data['age'])]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


## Estratégia de tratamento de valores nulos

In [70]:
# importação da biblioteca de pré-processamento
# Essa biblioteca irá colocar um valor de entrada, ou seja, uma substituição
# Será o responsável por realizar o tratamento dessas informações nulas
from sklearn.impute import SimpleImputer

#imputer = SimpleImputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
# Axis = 0 igual a coluna | 1 as linhas
# Esta forma é igual a de cima
imputer = SimpleImputer()

# Aqui irá realizar o tratamento em todas as linhas e em todas as colunas dos previsores
imputer = imputer.fit(X[:, 0:3])

X[:, 0:3] = imputer.transform(X[:, 0:3])

In [77]:
## Adicionar uma linha aqui informando se foi realizado o tratamento dos valores nulos 
# Indicação das colunas:
# [0] = Renda
# [1] = Idade
# [2] = Emprestimo

# Notamos que abaixo não possui mais nenhum valor negativo
pd.DataFrame(X).isnull().sum()

0    0
1    0
2    0
dtype: int64

## Verificação de escala entre as features (Atributos previsores)

In [78]:
# Exibição das primeiras 10 linhas abaixo
# Indicação das colunas:
# [0] = Renda
# [1] = Idade
# [2] = Emprestimo

pd.DataFrame(X).head(10)

# Pode-se verificar que entre as colunas de Renda e idade há uma diferença muito grande entre as escalas
# Ou seja, escalas distintas
# Podemos verificar pelo seguinte exemplo considerando os dois primeiros registros:
# OBS: Levando em consideração que iremos utilizar um algoritmo baseado em distância como o KNN
# 66155 - 34415 = 31740 | 59 - 48 = 11 
# De acordo com o que foi dito acima, o KNN irá considerar que a renda é mais importante que a idade
# Devido a possuir valores maiores de escala em relação à idade
# Porém, não é isso que queremos. Pra isso temos que dá a mesma importância da renda pra idade

,0,1,2
0,66155.925095,59.017015,8106.532131
1,34415.153966,48.117153,6564.745018
2,57317.170063,63.108049,8020.953296
3,42709.534201,45.751972,6103.642260
4,66952.688845,18.584336,8770.099235
5,24904.064140,57.471607,15.498598
6,48430.359613,26.809132,5722.581981
7,24500.141984,32.897548,2971.003310
8,40654.892537,55.496853,4755.825280
9,25075.872771,39.776378,1409.230371


### Para balancear as escalas deverá ser usado o escalonamento dos atributos (Feature Scalling)

## Escalonamento de atributos

In [79]:
# Import da lib para o pré-processamento chamada de Padronização

from sklearn.preprocessing import StandardScaler

# Não é necessário passar nenhum parâmetro
scaler = StandardScaler()

X = scaler.fit_transform(X)

pd.DataFrame(X).head(10)

# Agora não existe mais aquela diferença entre as escalas das features
# Dessa forma, os algoritmos que possuem a estratégia baseada em distância euclidiana
# Não deverá mais dá importância a somente n features, mas sim a todas.

,0,1,2
0,1.453934,1.365380,1.202819
1,-0.762176,0.542659,0.696427
2,0.836821,1.674171,1.174711
3,-0.183070,0.364136,0.544980
4,1.509563,-1.686475,1.420765
5,-1.426230,1.248733,-1.454641
6,0.216352,-1.065668,0.419823
7,-1.454432,-0.606115,-0.483920
8,-0.326523,1.099678,0.102296
9,-1.414235,-0.086902,-0.996877
